# IINTS-AF in 5 Minutes

This notebook benchmarks a custom AI controller (PID) against a standard pump on a virtual patient.

In [ ]:
# If you're running in Colab, install the SDK first
# !pip -q install -i https://test.pypi.org/simple/ iints-sdk-python35

In [ ]:
import pandas as pd
from iints.core.patient.models import PatientModel
from iints.core.algorithms.pid_controller import PIDController
from iints.core.algorithms.standard_pump_algo import StandardPumpAlgorithm
from iints.core.simulator import Simulator, StressEvent
from iints.analysis.clinical_metrics import ClinicalMetricsCalculator

In [ ]:
def run_algorithm(algorithm, meals, duration_minutes=24 * 60):
    patient = PatientModel(initial_glucose=120.0)
    simulator = Simulator(
        patient_model=patient,
        algorithm=algorithm,
        time_step=5,
        seed=42,
        enable_profiling=True,
    )

    for meal in meals:
        simulator.add_stress_event(
            StressEvent(start_time=meal["time"], event_type="meal", value=meal["carbs"])
        )

    results_df, safety_report = simulator.run_batch(duration_minutes=duration_minutes)
    return results_df, safety_report

meals = [
    {"time": 8 * 60, "carbs": 60},
    {"time": 13 * 60, "carbs": 70},
    {"time": 19 * 60, "carbs": 80},
]

pid_df, pid_report = run_algorithm(PIDController(), meals)
pump_df, pump_report = run_algorithm(StandardPumpAlgorithm(), meals)

calculator = ClinicalMetricsCalculator()
pid_metrics = calculator.calculate(pid_df["glucose_actual_mgdl"], duration_hours=24).to_dict()
pump_metrics = calculator.calculate(pump_df["glucose_actual_mgdl"], duration_hours=24).to_dict()

summary = pd.DataFrame([
    {"algorithm": "PID", "tir_70_180": pid_metrics["tir_70_180"], "cv": pid_metrics["cv"], "mean_glucose": pid_metrics["mean_glucose"]},
    {"algorithm": "Standard Pump", "tir_70_180": pump_metrics["tir_70_180"], "cv": pump_metrics["cv"], "mean_glucose": pump_metrics["mean_glucose"]},
])

summary

In [ ]:
# Inspect performance profiling (mean latency in ms)
pid_report["performance_report"]["algorithm_latency_ms"]